In [28]:
import rlcms.composites as composites
import ee
import geemap

ee.Initialize()

In [32]:
# some args we'll use in both composites
start_date = '2019-01-01'
end_date = '2020-12-31'
reducer = 'mean' # min, max, ee.Reducer objects
composite_mode = 'annual' # 'annual'
season=[1,2,3,4,5] # required if composite_mode=='seasonal'

binga_polys = ee.FeatureCollection("projects/wwf-sig/assets/kaza-lc/reference_data/BingaDummyReferencePolys")
ca_poly = ee.Geometry.Polygon([[-120.70162384826843,36.658181801020476],
                               [-120.65218537170593,36.658181801020476],
                               [-120.65218537170593,36.68461734040718],
                               [-120.70162384826843,36.68461734040718],
                               [-120.70162384826843,36.658181801020476]])
colombia_polys = ee.Geometry.MultiPolygon([[[-74.27,3.19],
                                         [-74.26,3.19],
                                         [-74.26,3.20],
                                         [-74.27,3.20],
                                         [-74.27,3.19]],
                                        
                                        [[-74.21,3.16],
                                         [-74.20,3.16],
                                         [-74.20,3.17],
                                         [-74.21,3.17],
                                         [-74.21,3.16]],
                                        
                                        [[-74.15,3.14],
                                         [-74.14,3.14],
                                         [-74.14,3.15],
                                         [-74.15,3.15],
                                         [-74.15,3.14]]])
s1composite = composites.composite(dataset='Sentinel1',
                                 region=colombia_polys,
                                 start_date=start_date,
                                 end_date=end_date,
                                 multi_poly=True, # if region is multiple polygons, set this flag
                                 composite_mode=composite_mode,
                                 season=season,
                                 reducer=reducer, 
                                 harmonicsOptions={'VV':{'start':1,'end':180},
                                                   'VH':{'start':1,'end':180}})

s2composite = composites.composite(dataset='Sentinel2',
                                 region=colombia_polys,
                                 start_date=start_date,
                                 end_date=end_date,
                                 multi_poly=True,
                                 indices=['IBI','SAVI'],
                                 composite_mode=composite_mode,
                                 season=season,
                                 reducer=reducer,
                                 harmonicsOptions={'nir':{'start':1,'end':180}},
                                 addJRC=True,
                                 addTopography=True)


all_composite = s2composite.addBands(s1composite)
print(all_composite.bandNames().getInfo())

['t0_blue_mean', 't0_green_mean', 't0_red_mean', 't0_nir_mean', 't0_swir1_mean', 't0_swir2_mean', 't0_IBI_mean', 't0_SAVI_mean', 't1_blue_mean', 't1_green_mean', 't1_red_mean', 't1_nir_mean', 't1_swir1_mean', 't1_swir2_mean', 't1_IBI_mean', 't1_SAVI_mean', 'nir_phase', 'nir_amplitude', 'elevation', 'slope', 'aspect', 'eastness', 'northness', 't0_VV_mean', 't0_VH_mean', 't0_angle_mean', 't1_VV_mean', 't1_VH_mean', 't1_angle_mean', 'VH_phase', 'VH_amplitude', 'VV_phase', 'VV_amplitude']


In [33]:
Map = geemap.Map()

s2_params = {'bands':['red_mean','blue_mean','green_mean'],
              'min':0,'max':1500,
              }
s2_params_multi_year = {'bands':['t0_red_mean','t0_blue_mean','t0_green_mean'],
              'min':0,'max':1500,
              }
s1_params = {'bands':'VV_mean','min':-22,'max':2}
s1_params_multi_year = {'bands':'t0_VV_mean','min':-22,'max':2}
Map.addLayer(binga_polys,{},'binga pols FC')
Map.addLayer(ca_poly,{},'single poly geo')
Map.addLayer(colombia_polys,{},'multi poly geometry')
Map.addLayer(all_composite,s2_params_multi_year,'all_composite rgb')
Map.addLayer(all_composite,s1_params_multi_year,'all_composite VV')
Map.addLayer(s1composite,s1_params_multi_year,'s1composite VV')

Map.centerObject(colombia_polys,12)

Map

Map(center=[3.1950000040192426, -74.26499999997142], controls=(WidgetControl(options=['position', 'transparent…